In [85]:
# Start your code here!
import os
import pandas as pd
from openai import OpenAI

# Instantiate an API client
client = OpenAI(api_key='OPEN_AI_KEY')

# Continue coding here
nasdaq100 = pd.read_csv('nasdaq100.csv')
nasdaq100_price_change = pd.read_csv("nasdaq100_price_change.csv")

In [94]:
# Add a column called "ytd" containing year to date (YTD) performance for each company.
nasdaq100 = pd.merge(nasdaq100, nasdaq100_price_change[['symbol', 'ytd']], on='symbol', how='left')

In [95]:
nasdaq100

,symbol,name,headQuarter,dateFirstAdded,cik,founded,ytd_x,sector,ytd_y
0,AAPL,Apple Inc.,"Cupertino, CA",NaN,320193,1976-04-01,42.99992,Apple Inc.,42.99992
1,ABNB,Airbnb,"San Francisco, CA",NaN,1559720,2008-08-01,68.66902,Airbnb,68.66902
2,ADBE,Adobe Inc.,"San Jose, CA",NaN,796343,1982-12-01,57.22723,Adobe Inc.,57.22723
3,ADI,Analog Devices,"Wilmington, MA",NaN,6281,1965-01-01,17.02062,Analog Devices,17.02062
4,ADP,ADP,"Roseland, NJ",NaN,8670,1949-01-01,5.53732,ADP,5.53732
...,...,...,...,...,...,...,...,...,...
96,WBD,Warner Bros. Discovery,"New York, NY",NaN,1437107,2022-04-08,51.67715,Warner Bros. Discovery,51.67715
97,WDAY,"Workday, Inc.","Pleasanton, CA",NaN,1327811,2005-03-01,37.90965,"Workday, Inc.",37.90965
98,XEL,Xcel Energy,"Minneapolis, MN",NaN,72903,1909-06-17,-14.71386,Xcel Energy,-14.71386
99,ZM,Zoom Video Communications,"San Jose, CA",NaN,1585521,2011-04-21,3.00030,Zoom Video Communications,3.00030


### Function to get the sector name for a given company

In [88]:
def get_sector_name(company_name):
    response = client.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=f'''
      Classify the sector of the {company_name} from the classes delimited by triple backticks:
      ```Technology, Consumer Cyclical, Industrials, Utilities, Healthcare, Communication, Energy, Consumer Defensive, Real Estate, Financial```
      ''',
      max_tokens=100,
      temperature=0.2
    )
    sector_name = response.choices[0].text
    return sector_name

In [90]:
nasdaq100['sector'] = nasdaq100['name'].apply(lambda x: get_sector_name(x))

### Get stock recommendations

In [45]:
def get_stock_recommendations(nasdaq100_info):
    instruction = f'''
    You will be provided will nasdaq100 data delimited by triple backticks. Recommend the three best sectors and per sector three or more companies.
    '''
    
    output_format = f'''
    Use the following format for the Output:
    - Sector: <First recommendation sector>, Companies: <three companies per sector>
    - Sector: <Second recommendation sector>, Companies: <three companies per sector>
    - Sector: <Third recommendation sector>, Companies: <three companies per sector>
    '''
    
    prompt = instruction + output_format + f"```{nasdaq100_info}```"
    
    response = client.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=prompt,
      temperature=0.2,
      max_tokens=2000
    )
    stock_recommendations = response.choices[0].text
    return stock_recommendations

In [92]:
nasdaq_filtered_column = nasdaq100[['name', 'ytd','sector']]

In [49]:
stock_recommendations = get_stock_recommendations(nasdaq_filtered_column.to_string())

In [52]:
print(stock_recommendations)


- Sector: Technology, Companies: Apple Inc., Adobe Inc., Analog Devices
- Sector: Real Estate, Companies: Airbnb
- Sector: Technology, Companies: ADP
